In [ ]:
#from app.DataLoader import DataLoader
from app.DataLoader import DataLoader
from app.DataSpliter import DataSplitter, GranularDataSplitter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
loader = DataLoader("data/2024-02-25/HUAWEI_MATE/final_df.csv")
data = loader.load_transform_data()
data

In [ ]:
spliter = DataSplitter(data)

In [ ]:
# splited_data = spliter.split_data(start_time="12:44:30",end_time="12:46:45",offset="-0:00:02.5")
# splited_data

In [ ]:
# ! by frame
# ! also it is right leg so i look on that
# ! so the point we are looking is when skier is going straight downhill
curves2 = [
    "12:44:30.2",
    "12:44:33.9",
    "12:44:37.4",
    "12:44:40.6",
    "12:44:41.8",
    "12:44:46.6",
    "12:44:56.7",
    "12:45:00.7",
    "12:45:03.4",
    "12:45:07.4",
    "12:45:10.3",
    "12:45:16.3",
    "12:45:19.8",
    "12:45:23.9",
    "12:45:28.4",
    "12:45:31.9",
    "12:45:37.1",
    "12:45:44.5",
    "12:45:49.9",
    "12:45:53.2",
    "12:45:58.4",
    "12:46:00.9",
    "12:46:06.3",
    "12:46:10.4",
    "12:46:13.3",
    "12:46:15.4",
    "12:46:17.9",
    "12:46:20.3",
    "12:46:22.8",
    "12:46:25.7",
    "12:46:31.5",
    "12:46:34.0",
    "12:46:36.9",
    "12:46:43.1",

]

In [ ]:
curves_R = []
curves_L = []
for i in range(len(curves2)):
    if i % 2 == 0:
        curves_L.append(curves2[i])
    else:
        curves_R.append(curves2[i])

In [ ]:
curves_R

In [ ]:
curves_L

In [ ]:
labeled_data = spliter.split_label_data(start_time="12:44:30",end_time="12:46:45",offset="-0:00:01.6", curves_L=curves_L, curves_R=curves_R)

In [ ]:
#select only rows from labeled_data where Curve value is true with usage of isin
# labeled_data[labeled_data["Curve"].isin([True])]
# it also depend by leg


In [ ]:
from app.visualization_utils import draw_plotly


In [ ]:
draw_plotly(labeled_data, "Orientation")

In [ ]:
#find local extremas in the - also needs to be unpacked

In [ ]:
temp = labeled_data.copy()
temp

### Podzielenie danych 

In [ ]:
Granular_spliter = GranularDataSplitter(temp)

In [ ]:
Granular_spliter.split_into_granular()

In [ ]:
df = Granular_spliter.granular_data["Orientation"].copy()

### Smoothing 

In [ ]:
sensor_value = "roll"

In [ ]:
#create a dataframe with only one column sensor_value not a series
#df_smooth = df[[sensor_value]]


In [ ]:
from app.transformers.smoother import Smoother
data_smoother = Smoother(df)

In [ ]:
df

In [ ]:
draw_plotly(data_smoother.smooth_data_exponential(span=10, columns=[sensor_value]), "Orientation", granular=True)


In [ ]:
draw_plotly(data_smoother.smooth_data_rolling_mean(window_size=10, columns=[sensor_value]), "Orientation", granular=True)


In [ ]:
draw_plotly(data_smoother.smooth_data_backward_ma(window_size=10, columns=[sensor_value]), "Orientation", granular=True)


In [ ]:
smoothed_roll = data_smoother.smooth_data_backward_ma(window_size=10, columns=[sensor_value])

### Gradient

In [ ]:
smoothed_roll

In [ ]:
# () gradient descent with momentum? min max approach
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Literal
# Przykład wczytania danych do DataFrame
# df = pd.read_csv('twoje_dane.csv')
# Zakładam, że kolumna "roll" istnieje i index jest poprawny
def plot_positions(series, positions, step, curves = None, find_maxima = None):
    plt.figure(figsize=(12, 6))
    plt.plot(series, label='Series', color='blue')
    plt.scatter(positions, series[positions], color='red', label='Positions')
    if curves is not None:
        plt.scatter(curves, series[curves], color='green', label='Curves')
    plt.xlabel('Index')
    plt.ylabel('Value')
    if find_maxima is None:
        plt.title('Positions at step {}'.format(step))
    else:
        extremum = "maxima" if find_maxima else "minima"
        plt.title('Positions at step {} for {}'.format(step, extremum))

    plt.legend()
    plt.show()
def get_curve_points(df:pd.DataFrame, turn: Literal["L", "R"] = None):
    if turn:
        return np.where(df[f'Curve'] == turn)[0].tolist()
    return np.where(df['Curve'])[0].tolist()
    # Funkcja gradient descent do znajdowania ekstremów

def gradient_descent(df, series, normalized_series, normalized_positions,  start_indices, learning_rate, steps, momentum, find_maxima, printing):
    velocities = [0 for _ in start_indices]
    for step in range(steps):
        for i, idx in enumerate(normalized_positions):
            ahead_idx = int(min(1, idx + 1/(len(series) - 1)) * (len(series) - 1))
            behind_idx = int(max(0, idx - 1/(len(series) - 1)) * (len(series) - 1))

            grad = (normalized_series[ahead_idx] - normalized_series[behind_idx]) / 2

            # Update velocity and position with momentum
            if find_maxima:
                velocities[i] = momentum * velocities[i] + learning_rate * grad  # Gradient ascent for maxima
            else:
                velocities[i] = momentum * velocities[i] - learning_rate * grad  # Gradient descent for minima

            new_idx = idx + velocities[i]
            new_idx = max(0, min(1, new_idx))  # Ensure new_idx stays within 0-1

            normalized_positions[i] = new_idx
            #print for step for step
            #print(f'point: {i}, ahed_idx: {ahead_idx}, behind_idx: {behind_idx}, grad: {grad}, new_idx: {new_idx}, velocities: {velocities[i]}')
        positions = [int(idx * (len(series) - 1)) for idx in normalized_positions]
        if step % 10 == 0 and printing:
            plot_positions(series, positions, step, get_curve_points(df))
    # print(set(history))
    positions = [int(idx * (len(series) - 1)) for idx in normalized_positions]
    plot_positions(series, positions, "final", get_curve_points(df), find_maxima)
    return positions

def gradient_descent_full(df, start_indices, learning_rate=0.01, steps=1000, momentum=0.98, printing = False):
    series = df['roll'].values

    positions = start_indices.copy()
    #normalize series data to 0-1
    normalized_series = (series - series.min()) / (series.max() - series.min())

    # Normalize start_indices to 0-1
    normalized_positions = [idx / (len(series) - 1) for idx in start_indices]

    #initial plot
    plot_positions(series, positions, "beginning", get_curve_points(df))

    min_positions = gradient_descent(df, series, normalized_series, normalized_positions,  start_indices, learning_rate, steps, momentum, False, printing)
    max_positions = gradient_descent(df, series, normalized_series, normalized_positions,  start_indices, learning_rate, steps, momentum, True, printing)

    return min_positions, max_positions



# Wybieramy kolumnę "roll" z DataFrame

# Krok 1: Losowe wybieranie punktów początkowych
#np.random.seed(42)  # Ustawienie ziarna losowości dla powtarzalności wyników
num_points = 1000
start_indices =np.random.randint(1, len(smoothed_roll)-1, size=num_points)  #np.array([800])

# Krok 2: Znalezienie ekstremów przy użyciu gradient descent
final_positions_min, final_positions_maximums = gradient_descent_full(smoothed_roll, start_indices, printing=False)

# # Krok 3: Filtracja najważniejszych ekstremów
# unique_positions, counts = np.unique(final_positions, return_counts=True)
# threshold = np.percentile(counts, 1)  # Przykładowy próg
# important_extrema_indices = unique_positions[counts >= threshold]

# Wynik - DataFrame z najważniejszymi ekstremami
#important_extrema_smoothed_roll = smoothed_roll.iloc[final_positions]


In [ ]:
# # Krok 3: Filtracja najważniejszych ekstremów

def filter_extremes(final_positions):
    unique_positions, counts = np.unique(final_positions, return_counts=True)
    threshold = np.percentile(counts, 75)  # Przykładowy próg
    important_extrema_indices = unique_positions[counts >= threshold]
    return important_extrema_indices

# unique_positions, counts = np.unique(final_positions_min, return_counts=True)
# threshold = np.percentile(counts, 75)  # Przykładowy próg
# important_extrema_indices = unique_positions[counts >= threshold]

# Wynik - DataFrame z najważniejszymi ekstremami
#important_extrema_smoothed_roll = smoothed_roll.iloc[final_positions]
important_min = filter_extremes(final_positions_min)
important_max = filter_extremes(final_positions_maximums)


In [ ]:
important_min

In [ ]:
important_max

In [ ]:
plot_positions(smoothed_roll['roll'].values, important_min, "mins", get_curve_points(smoothed_roll, "L"))

In [ ]:
plot_positions(smoothed_roll['roll'].values, important_max, "max", get_curve_points(smoothed_roll, "R"))

In [ ]:
len(np.where(smoothed_roll['Curve'].isin(["R"]))[0])

In [ ]:
# ! Later need to be adjusted /left/right turs for now there is just a "turn"
def get_distance(df, positions, turn ):

    curves = np.where(smoothed_roll['Curve'].isin([turn]))[0]
    positions = np.sort(positions)
    curves = np.sort(curves)

    distances = []

    for i in range(len(positions)):
        j = 0
        while j < len(curves)-1 and abs(positions[i] - curves[j]) > abs(positions[i] - curves[j+1]):
            j += 1
        distances.append(abs(positions[i] - curves[j]))

    return distances


In [ ]:
distances_R = get_distance(smoothed_roll, important_max, "R")
distances_L = get_distance(smoothed_roll, important_min, "L")

In [ ]:
distances_R

In [ ]:
distances_L

In [ ]:
distances_L

In [ ]:
distances

In [ ]:
type(df['roll'])

### argrelextrema

In [ ]:
from scipy.signal import argrelextrema
local_maxima_indices = argrelextrema(smoothed_roll[sensor_value].values, np.greater)[0]
local_maxima = smoothed_roll.iloc[local_maxima_indices]

# Znalezienie lokalnych minimów
local_minima_indices = argrelextrema(smoothed_roll[sensor_value].values, np.less)[0]
local_minima = smoothed_roll.iloc[local_minima_indices]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(smoothed_roll.index, smoothed_roll[sensor_value], label=sensor_value, color='blue')
plt.scatter(local_maxima.index, local_maxima[sensor_value], color='red', label='Local Maxima', zorder=5)
plt.scatter(local_minima.index, local_minima[sensor_value], color='green', label='Local Minima', zorder=5)
plt.title(f'{sensor_value} with Local Extrema')
plt.xlabel('Time [s]')
plt.ylabel(sensor_value)
plt.legend()
plt.grid(True)
plt.show()